EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/ca_real_estate.csv')

In [3]:
df = df.drop(columns=['Province'])

FEATURE ENCODING

In [4]:
df = pd.get_dummies(df, columns=['City', 'Type'], drop_first=True).astype(int)

In [5]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error

# Define features and target
X = df.drop(columns=['Price'])  # Features
y = df['Price']  # Target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FEATURE SCALING

In [6]:
from sklearn.preprocessing import StandardScaler
import joblib

numerical_features = ["SqFt", "Lot_Area", "Year_Built", "Bedrooms", "Bathrooms", "Garage"]
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])  # Use same scaler for test set

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [7]:
param_grid = {
    'n_estimators': [100, 300, 500], 
    'learning_rate': [0.01, 0.1, 0.2], 
    'max_depth': [3, 5, 7],  
    'min_child_weight': [1, 3, 5], 
    'subsample': [0.8, 1.0],  
    'colsample_bytree': [0.8, 1.0]  
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all CPU cores
)

grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)


Fitting 3 folds for each of 324 candidates, totalling 972 fits
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 0.8}
